##### Copyright 2021 The TensorFlow Hub Authors.

Licensed under the Apache License, Version 2.0 (the "License");

In [ ]:
#@title Copyright 2021 The TensorFlow Hub Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://tensorflow.google.cn/hub/tutorials/wav2vec2_saved_model_finetuning"><img src="https://tensorflow.google.cn/images/tf_logo_32px.png">View on TensorFlow.org</a> </td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/zh-cn/hub/tutorials/wav2vec2_saved_model_finetuning.ipynb"><img src="https://tensorflow.google.cn/images/colab_logo_32px.png">在 Google Colab 中运行</a></td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/zh-cn/hub/tutorials/wav2vec2_saved_model_finetuning.ipynb"><img src="https://tensorflow.google.cn/images/GitHub-Mark-32px.png">在 GitHub 上查看源代码</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/zh-cn/hub/tutorials/wav2vec2_saved_model_finetuning.ipynb"><img src="https://tensorflow.google.cn/images/download_logo_32px.png">下载笔记本</a>
</td>
  <td>     <a href="https://tfhub.dev/vasudevgupta7/wav2vec2/1"><img src="https://tensorflow.google.cn/images/hub_logo_32px.png">查看 TF Hub 模型 </a> </td>
</table>

# 使用 LM 头微调 Wav2Vec2

在本笔记本中，我们将从 [TFHub](https://tfhub.dev) 加载预训练的 wav2vec2 模型，然后通过在预训练模型上附加语言建模头 (LM) 来使用 [LibriSpeech 数据集](https://huggingface.co/datasets/librispeech_asr)对该模型进行微调。基本任务是构建一个**自动语音识别**模型，即，提供一些语音，该模型应该能够将其转录成文本。

## 设置

在运行本笔记本之前，请确保您处于 GPU 运行时 (`Runtime` &gt; `Change runtime type` &gt; `GPU`)。以下单元将安装 [`gsoc-wav2vec2`](https://github.com/vasudevgupta7/gsoc-wav2vec2) 包及其依赖项。

In [ ]:
!pip3 install -q git+https://github.com/vasudevgupta7/gsoc-wav2vec2@main
!sudo apt-get install -y libsndfile1-dev
!pip3 install -q SoundFile

## 使用 `TFHub` 设置模型

我们先导入一些库/模块。

In [ ]:
import os

import tensorflow as tf
import tensorflow_hub as hub
from wav2vec2 import Wav2Vec2Config

config = Wav2Vec2Config()

print("TF version:", tf.__version__)

首先，我们将从 TFHub 下载模型，并使用 [`hub.KerasLayer`](https://tensorflow.google.cn/hub/api_docs/python/hub/KerasLayer) 包装模型签名，以便能够像使用任何其他 Keras 层一样使用此模型。幸运的是，只需 1 行代码，`hub.KerasLayer` 就可以完成这两项操作。

**注：**当使用 `hub.KerasLayer` 加载模型时，模型会变得有点不透明，但有时我们需要对模型进行更精细的控制，然后可以使用 `tf.keras.models.load_model(...)` 加载模型。

In [ ]:
pretrained_layer = hub.KerasLayer("https://tfhub.dev/vasudevgupta7/wav2vec2/1", trainable=True)

如果您对模型导出脚本感兴趣，可以参考此[脚本](https://github.com/vasudevgupta7/gsoc-wav2vec2/blob/main/src/export2hub.py)。对象 `pretrained_layer` 是 [`Wav2Vec2Model`](https://github.com/vasudevgupta7/gsoc-wav2vec2/blob/main/src/wav2vec2/modeling.py) 的冻结版本。这些预训练权重是使用[此脚本](https://github.com/vasudevgupta7/gsoc-wav2vec2/blob/main/src/convert_torch_to_tf.py)从 HuggingFace PyTorch [预训练权重](https://huggingface.co/facebook/wav2vec2-base)转换而来的。

最初，wav2vec2 是使用遮罩语言建模方法进行预训练的，目的是确定遮罩时间步长的真实量化潜在语音表示。有关训练目标的更多信息，可以参阅论文 [wav2vec 2.0: A Framework for Self-Supervised Learning of Speech Representations](https://arxiv.org/abs/2006.11477)。

现在，我们将定义一些常量和超参数，它们将在接下来的几个单元中用到。`AUDIO_MAXLEN` 有意设置为 `246000`，因为模型签名仅接受 `246000` 的静态序列长度。

In [ ]:
AUDIO_MAXLEN = 246000
LABEL_MAXLEN = 256
BATCH_SIZE = 2

在以下单元中，我们将使用 [Keras 的函数 API](https://tensorflow.google.cn/guide/keras/functional) 包装 `pretrained_layer` 和一个密集层（LM 头）。

In [ ]:
inputs = tf.keras.Input(shape=(AUDIO_MAXLEN,))
hidden_states = pretrained_layer(inputs)
outputs = tf.keras.layers.Dense(config.vocab_size)(hidden_states)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

上面定义的密集层的输出维度为 `vocab_size`，因为我们希望在每个时间步长都预测词汇表中每个标记的概率。

## 设置训练状态

在 TensorFlow 中，只有第一次调用 `model.call` 或 `model.build` 时才会构建模型权重，因此下面的单元将为我们构建模型权重。此外，我们将运行 `model.summary()` 来检查可训练参数的总数。

In [ ]:
model(tf.random.uniform(shape=(BATCH_SIZE, AUDIO_MAXLEN)))
model.summary()

现在，我们需要定义 `loss_fn` 和优化器才能训练模型。下面的单元将执行此操作。为简单起见，我们将使用 `Adam` 优。`CTCLoss`是一个常见的损失类型，用于输入子部分无法轻易与输出子部分对齐的任务（如 `ASR`）。您可以从这篇精彩的[博客文章](https://distill.pub/2017/ctc/)中了解有关 CTC 损失的更多信息。

`CTCLoss`（来自 [`gsoc-wav2vec2`](https://github.com/vasudevgupta7/gsoc-wav2vec2) 包）接受 3 个参数：`config`、`model_input_shape` 和 `division_factor`。如果 `division_factor=1`，那么损失将简单地相加，因此相应地传递 `division_factor` 可获得批次的平均值。

In [ ]:
from wav2vec2 import CTCLoss

LEARNING_RATE = 5e-5

loss_fn = CTCLoss(config, (BATCH_SIZE, AUDIO_MAXLEN), division_factor=BATCH_SIZE)
optimizer = tf.keras.optimizers.Adam(LEARNING_RATE)

## 加载和预处理数据

现在让我们从[官方网站](http://www.openslr.org/12)下载 LibriSpeech 数据集并进行设置。

In [ ]:
!wget https://www.openslr.org/resources/12/dev-clean.tar.gz -P ./data/train/
!tar -xf ./data/train/dev-clean.tar.gz -C ./data/train/

**注：**我们将使用 `dev-clean` 配置，因为本笔记本只为演示之用，因此只需要少量数据。完整的训练数据可以轻易地从 [LibriSpeech 网站](http://www.openslr.org/12)下载。

In [ ]:
ls ./data/train/

我们的数据集位于 LibriSpeech 目录中。让我们浏览一下这些文件。

In [ ]:
data_dir = "./data/train/LibriSpeech/dev-clean/2428/83705/"
all_files = os.listdir(data_dir)

flac_files = [f for f in all_files if f.endswith(".flac")]
txt_files = [f for f in all_files if f.endswith(".txt")]

print("Transcription files:", txt_files, "\nSound files:", flac_files)

每个子目录都有许多 `.flac` 文件和一个 `.txt` 文件。这个 `.txt` 文件包含该子目录中存在的所有语音样本（即 `.flac` 文件）的文本转录。

我们可以按如下方式加载此文本数据：

In [ ]:
def read_txt_file(f):
  with open(f, "r") as f:
    samples = f.read().split("\n")
    samples = {s.split()[0]: " ".join(s.split()[1:]) for s in samples if len(s.split()) > 2}
  return samples

同样，我们将定义一个从 `.flac` 文件加载语音样本的函数。

`REQUIRED_SAMPLE_RATE` 设置为 `16000`，因为 wav2vec2 是使用 `16K` 频率进行预训练的，建议在数据分布不会因频率而发生任何重大变化的情况下进行微调。

In [ ]:
import soundfile as sf

REQUIRED_SAMPLE_RATE = 16000

def read_flac_file(file_path):
  with open(file_path, "rb") as f:
      audio, sample_rate = sf.read(f)
  if sample_rate != REQUIRED_SAMPLE_RATE:
      raise ValueError(
          f"sample rate (={sample_rate}) of your files must be {REQUIRED_SAMPLE_RATE}"
      )
  file_id = os.path.split(file_path)[-1][:-len(".flac")]
  return {file_id: audio}

现在，我们将挑选一些随机样本并尝试将它们可视化。

In [ ]:
from IPython.display import Audio
import random

file_id = random.choice([f[:-len(".flac")] for f in flac_files])
flac_file_path, txt_file_path = os.path.join(data_dir, f"{file_id}.flac"), os.path.join(data_dir, "2428-83705.trans.txt")

print("Text Transcription:", read_txt_file(txt_file_path)[file_id], "\nAudio:")
Audio(filename=flac_file_path)

现在，我们将结合所有语音和文本样本，并为该目的定义函数（在下一个单元中）。

In [ ]:
def fetch_sound_text_mapping(data_dir):
  all_files = os.listdir(data_dir)

  flac_files = [os.path.join(data_dir, f) for f in all_files if f.endswith(".flac")]
  txt_files = [os.path.join(data_dir, f) for f in all_files if f.endswith(".txt")]

  txt_samples = {}
  for f in txt_files:
    txt_samples.update(read_txt_file(f))

  speech_samples = {}
  for f in flac_files:
    speech_samples.update(read_flac_file(f))

  assert len(txt_samples) == len(speech_samples)

  samples = [(speech_samples[file_id], txt_samples[file_id]) for file_id in speech_samples.keys() if len(speech_samples[file_id]) < AUDIO_MAXLEN]
  return samples

是时候看看几个样本了...

In [ ]:
samples = fetch_sound_text_mapping(data_dir)
samples[:5]

注：我们将此数据加载到内存中是因为我们在本笔记本中只处理少量数据集。但是对于基于完整数据集（约 300 GB）的训练，您必须延迟加载数据。您可以参考[此脚本](https://github.com/vasudevgupta7/gsoc-wav2vec2/blob/main/src/data_utils.py)以了解更多相关信息。

现在，让我们对数据进行预处理！！！

我们先使用 `gsoc-wav2vec2` 包定义分词器 (tokenizer) 和处理器 (processor)。然后，我们将进行非常简单的预处理。`processor` 将根据帧轴对原始语音进行标准化，`tokenizer` 会将我们的模型输出转换为字符串（使用定义的词汇表）并负责删除特殊标记（取决于您的分词器配置）。

In [ ]:
from wav2vec2 import Wav2Vec2Processor
tokenizer = Wav2Vec2Processor(is_tokenizer=True)
processor = Wav2Vec2Processor(is_tokenizer=False)

def preprocess_text(text):
  label = tokenizer(text)
  return tf.constant(label, dtype=tf.int32)

def preprocess_speech(audio):
  audio = tf.constant(audio, dtype=tf.float32)
  return processor(tf.transpose(audio))

现在，我们将定义 python 生成器来调用我们在上面的单元中定义的预处理函数。

In [ ]:
def inputs_generator():
  for speech, text in samples:
    yield preprocess_speech(speech), preprocess_text(text)

## 设置`tf.data.Dataset`

以下单元将使用 `tf.data.Dataset` 对象的 `.from_generator(...)` 方法设置该对象。我们将使用上面的单元中定义的 `generator` 对象。

**注：**对于分布式训练（尤其是 TPU 上的训练），`.from_generator(...)` 目前不起作用，建议使用以 `.tfrecord` 格式存储的数据进行训练（注意：理想情况下，TFRecord 应存储在 GCS 存储分区中，以使 TPU 发挥最大作用）。

有关如何将 LibriSpeech 数据转换为 tfrecord 的更多详细信息，可以参考[此脚本](https://github.com/vasudevgupta7/gsoc-wav2vec2/blob/main/src/make_tfrecords.py)。

In [ ]:
output_signature = (
    tf.TensorSpec(shape=(None),  dtype=tf.float32),
    tf.TensorSpec(shape=(None), dtype=tf.int32),
)

dataset = tf.data.Dataset.from_generator(inputs_generator, output_signature=output_signature)

In [ ]:
BUFFER_SIZE = len(flac_files)
SEED = 42

dataset = dataset.shuffle(BUFFER_SIZE, seed=SEED)

我们会将数据集传递到多个批次中，因此让我们在以下单元中准备批次。现在，一个批次中的所有序列都应该填充到一个恒定的长度。为此，我们将使用 `.padded_batch(...)` 方法。

In [ ]:
dataset = dataset.padded_batch(BATCH_SIZE, padded_shapes=(AUDIO_MAXLEN, LABEL_MAXLEN), padding_values=(0.0, 0))

加速器（如 GPU/TPU）非常快，数据加载（和预处理）经常成为训练过程中的瓶颈，因为数据加载部分在 CPU 上进行。这会明显增加训练时间，尤其是涉及大量在线预处理或从 GCS 存储分区在线流式传输数据时。为了解决这些问题，`tf.data.Dataset` 提供了 `.prefetch(...)` 方法。当模型对当前批次进行预测（在 GPU/TPU 上）时，该方法有助于并行准备接下来的几个批次（在 CPU 上）。

In [ ]:
dataset = dataset.prefetch(tf.data.AUTOTUNE)

由于本笔记本用于演示目的，因此我们将采用第一个 `num_train_batches` 并只使用它进行训练。不过，我们鼓励您使用整个数据集进行训练。同样，我们将只评估 `num_val_batches`。

In [ ]:
num_train_batches = 10
num_val_batches = 4

train_dataset = dataset.take(num_train_batches)
val_dataset = dataset.skip(num_train_batches).take(num_val_batches)

## 模型训练

为了训练我们的模型，我们将在使用 `.compile(...)` 编译模型后直接调用 `.fit(...)` 方法。

In [ ]:
model.compile(optimizer, loss=loss_fn)

上面的单元将设置我们的训练状态。现在我们可以使用 `.fit(...)` 方法开始训练。

In [ ]:
history = model.fit(train_dataset, validation_data=val_dataset, epochs=3)
history.history

让我们使用 `.save(...)` 方法保存模型，以便稍后执行推断。您还可以按照 [TFHub 文档](https://tensorflow.google.cn/hub/publish)将此 SavedModel 导出到 TFHub。

In [ ]:
save_dir = "finetuned-wav2vec2"
model.save(save_dir, include_optimizer=False)

注：我们设置 `include_optimizer=False`，因为我们只想将此模型用于推断。

## 评估

现在我们将基于验证数据集计算词错误率

**词错误率** (WER) 是衡量自动语音识别系统性能的常用指标。WER 源自适用于单词级别的 Levenshtein 距离。词错误率的计算方法为：WER = (S + D + I) / N = (S + D + I) / (S + D + C)，其中 S 是替换次数，D 是删除次数，I 是插入次数，C 是正确词数，N 是参考中的字数 (N=S+D+C)。该值表示被错误预测的单词的百分比。

您可以参考[此论文](https://www.isca-speech.org/archive_v0/interspeech_2004/i04_2765.html)来了解有关 WER 的更多信息。

我们将使用 [HuggingFace 数据集](https://huggingface.co/docs/datasets/)库中的 `load_metric(...)` 函数。我们先使用 `pip` 安装 `datasets` 库，然后定义 `metric` 对象。

In [ ]:
!pip3 install -q datasets

from datasets import load_metric
metric = load_metric("wer")

In [ ]:
@tf.function(jit_compile=True)
def eval_fwd(batch):
  logits = model(batch, training=False)
  return tf.argmax(logits, axis=-1)

现在是时候根据验证数据进行评估了。

In [ ]:
from tqdm.auto import tqdm

for speech, labels in tqdm(val_dataset, total=num_val_batches):
    predictions  = eval_fwd(speech)
    predictions = [tokenizer.decode(pred) for pred in predictions.numpy().tolist()]
    references = [tokenizer.decode(label, group_tokens=False) for label in labels.numpy().tolist()]
    metric.add_batch(references=references, predictions=predictions)

我们使用 `tokenizer.decode(...)` 方法将预测和标签解码回文本，并将它们添加到 metric 中，以便稍后计算 `WER`。

现在，我们在以下单元中计算 metric：

In [ ]:
metric.compute()

**注：**这里的 metric 值没有任何意义，因为模型是使用非常少的数据训练的，而类似 ASR 的任务通常需要大量数据来学习从语音到文本的映射。您也许应该基于大数据进行训练以获得好的结果。本笔记本提供了一个微调预训练语音模型的模板。

## 推断

现在，我们对训练过程感到满意并将模型保存在 `save_dir` 中，我们将了解如何使用该模型进行推断。

首先，我们将使用 `tf.keras.models.load_model(...)` 加载模型。

In [ ]:
finetuned_model = tf.keras.models.load_model(save_dir)

让我们下载一些语音样本来进行推断。您也可以将以下样本替换为您的语音样本。

In [ ]:
!wget https://github.com/vasudevgupta7/gsoc-wav2vec2/raw/main/data/SA2.wav

现在，我们将使用 `soundfile.read(...)` 读取语音样本并将其填充到 `AUDIO_MAXLEN` 以符合模型签名。然后我们将使用 `Wav2Vec2Processor` 实例将该语音样本标准化并将其馈入模型。

In [ ]:
import numpy as np

speech, _ = sf.read("SA2.wav")
speech = np.pad(speech, (0, AUDIO_MAXLEN - len(speech)))
speech = tf.expand_dims(processor(tf.constant(speech)), 0)

outputs = finetuned_model(speech)
outputs

我们使用上面定义的 `Wav2Vec2tokenizer` 实例将数字解码回文本序列。

In [ ]:
predictions = tf.argmax(outputs, axis=-1)
predictions = [tokenizer.decode(pred) for pred in predictions.numpy().tolist()]
predictions

预测结果是相当随机的，因为本笔记本从未基于大数据训练模型（本笔记本并不适合进行完整训练）。如果您基于完整的 LibriSpeech 数据集训练此模型，您将获得良好的预测结果。

终于，我们来到本笔记本的结尾。但这并不是学习 TensorFlow 执行语音相关任务的结束，此[仓库](https://github.com/tulasiram58827/TTS_TFLite)包含一些更精彩的教程。如果您在本笔记本中遇到任何错误，请[在此处](https://github.com/vasudevgupta7/gsoc-wav2vec2/issues)创建问题。